## What I was Thinking

```2011-2018_monthly_refugee.csv``` contains data about the number of refugees taking shelter in what countries in what year. I was thinking of displaying a world map and depending on the number of refugees seen in the country in each, color each country accordingly. Somewhere down the line, we should add a scroll that allows the user to change the year and see the data change chronologically.

## What I Think Needs to be Done

Here is a tutorial for the plotly library: https://plot.ly/python/choropleth-maps/<br>
Here is a tutorial for creating a slider using plotly: https://plot.ly/python/gapminder-example/

If we do a world map, it would be better to use plotly since it has a world map already set for you. However, we need the country code to connect the map and the data in the csv, so... I don't really know what to do... Maybe Geopandas will actually be easier...

But if we do decide on plotly, we need to
1. find the country code and assign all rows a country code corresponding to the country
    1. https://www.kaggle.com/shep312/plotlycountrycodes
2. do some data wrangling and change some country names
    1. some country names have extra notes in the end, which we should delete
3. some values are represented with <b>*</b>, which means the data is confidential; those should be set to 0

## Some Things that Still Needs to be Done

1. Get rid of "trace 0" when hovering over countries
2. Maybe make the lines not hoverable since I can't hover over any European countries and have their country name displayed
    1. the lines are too dense over Europe
3. I decided against having endpoints of lines display something since it'll just display the country name again.

## Relevant Libraries

In [1]:
# import plotly.plotly as py
# from plotly.grid_objs import Grid, Column
# from plotly.tools import FigureFactory as FF 

import pandas as pd
import time
import re
import numpy as np

In [2]:
refugee_df = pd.read_csv('2011-2018_monthly_refugee.csv')
# refugee_df

In [3]:
# displays list of unique country names in the data frame
# refugee_df['Country / territory of asylum/residence'].unique()

Replace:
1. Czech Rep. --> Czech Republic
2. United Kingdom of Great Britain and Northern Ireland --> ??? (currently United Kingdom)
3. Rep. of Korea --> Korea, South
4. The former Yugoslav Rep. of Macedonia --> Macedonia
5. USA (INS/DHS) --> United States
6. USA (EOIR) --> United States
7. Serbia and Kosovo: S/RES/1244 (1999) --> Serbia

In [3]:
code_df = pd.read_csv('plotly_countries_and_codes.csv')
# code_df

In [4]:
locations_df = pd.read_csv('countries.csv')
# locations_df

In [6]:
# displays list of unique country names in the data frame
# code_df['COUNTRY'].unique()

In [5]:
# replacing values in refugee_df dataframe
new_refugee_df = refugee_df.replace({
    'Czech Rep.': 'Czech Republic',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Rep. of Korea': 'Korea, South',
    'The former Yugoslav Rep. of Macedonia': 'Macedonia',
    'USA (INS/DHS)': 'United States',
    'USA (EOIR)': 'United States',
    'Serbia and Kosovo: S/RES/1244 (1999)': 'Serbia'
})
# new_refugee_df

In [6]:
# joining two tables together
merged_df = new_refugee_df.set_index('Country / territory of asylum/residence').join(code_df.set_index('COUNTRY'))
merged_df = merged_df.reset_index()
# merged_df

In [7]:
# rename index column to 'Country'
merged_df.rename(columns={'index': 'Country'}, inplace=True)

# remove * and convert values to int values
merged_df['Value'] = merged_df['Value'].apply(lambda x : 0 if x == '*' else int(x))

# sum values and group each country by year
total_df = pd.DataFrame(merged_df.groupby(['Country','Year','CODE']).agg({'Value':np.sum})).reset_index()
# type(total_df)
# total_df

## Plotly

Need to make a plotly account and generate an api key

In [8]:
import plotly
import plotly.plotly as py
import plotly.offline as pyo
import plotly.graph_objs as go
plotly.__version__

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

plotly.tools.set_credentials_file(username='hwu43', api_key='8d3oItgJtnwYVfSQR7nL')

In [11]:
# # Add longitude and latitude values
# total_df = total_df.set_index('Country').join(locations_df.set_index('name'))
# total_df = total_df.reset_index()

# # rename index column to 'Country'
# total_df.rename(columns={'index': 'Country'}, inplace=True)

# # drop 'country' column which is the same as 'CODE'
# total_df = total_df.drop(columns=['country'], index=1)
# total_df

In [12]:
# from pandas.api.types import is_string_dtype
# from pandas.api.types import is_numeric_dtype
# # check type of latitude and longitude
# is_numeric_dtype(total_df['latitude'])

In [9]:
# find the total, just for reference/filler; not to be used in the final visualization
new_total_df = total_df.groupby(['Country', 'CODE']).sum()
new_total_df = new_total_df.reset_index()
# new_total_df

,Country,CODE,Year,Value
0,Albania,ALB,14105,2103
1,Australia,AUS,16116,719
2,Austria,AUT,16116,53865
3,Belgium,BEL,16116,23998
4,Bosnia and Herzegovina,BIH,14105,333
5,Bulgaria,BGR,16116,20986
6,Canada,CAN,16116,4748
7,Croatia,HRV,16116,1068
8,Cyprus,CYP,16116,6972
9,Czech Republic,CZE,16116,508


In [11]:
# merged_df = new_refugee_df.set_index('Country / territory of asylum/residence').join(code_df.set_index('COUNTRY'))
all_code_df = new_total_df.set_index('CODE').join(code_df.set_index('CODE'))
all_code_df = all_code_df.reset_index()
all_code_df

,CODE,Country,Year,Value,COUNTRY,GDP (BILLIONS)
0,ALB,Albania,14105,2103,Albania,13.40
1,AUS,Australia,16116,719,Australia,1483.00
2,AUT,Austria,16116,53865,Austria,436.10
3,BEL,Belgium,16116,23998,Belgium,527.80
4,BGR,Bulgaria,16116,20986,Bulgaria,55.08
5,BIH,Bosnia and Herzegovina,14105,333,Bosnia and Herzegovina,19.55
6,CAN,Canada,16116,4748,Canada,1794.00
7,CHE,Switzerland,16116,16954,Switzerland,679.00
8,CYP,Cyprus,16116,6972,Cyprus,21.34
9,CZE,Czech Republic,16116,508,Czech Republic,205.60


In [12]:
# Add longitude and latitude values
all_code_df = all_code_df.set_index('Country').join(locations_df.set_index('name'))
all_code_df = all_code_df.reset_index()

# rename index column to 'Country'
all_code_df.rename(columns={'index': 'Country'}, inplace=True)

# # drop 'country' column which is the same as 'CODE'
# all_code_df = all_code_df.drop(columns=['COUNTRY','country'], index=1)
# all_code_df

,Country,CODE,Year,Value,COUNTRY,GDP (BILLIONS),country,latitude,longitude
0,Albania,ALB,14105,2103,Albania,13.40,AL,41.153332,20.168331
1,Australia,AUS,16116,719,Australia,1483.00,AU,-25.274398,133.775136
2,Austria,AUT,16116,53865,Austria,436.10,AT,47.516231,14.550072
3,Belgium,BEL,16116,23998,Belgium,527.80,BE,50.503887,4.469936
4,Bulgaria,BGR,16116,20986,Bulgaria,55.08,BG,42.733883,25.485830
5,Bosnia and Herzegovina,BIH,14105,333,Bosnia and Herzegovina,19.55,BA,43.915886,17.679076
6,Canada,CAN,16116,4748,Canada,1794.00,CA,56.130366,-106.346771
7,Switzerland,CHE,16116,16954,Switzerland,679.00,CH,46.818188,8.227512
8,Cyprus,CYP,16116,6972,Cyprus,21.34,CY,35.126413,33.429859
9,Czech Republic,CZE,16116,508,Czech Republic,205.60,CZ,49.817492,15.472962


In [13]:
# rename Korea, South to South Korea
name_change = {
    'Korea, South': 'South Korea'
}

# renames the provinces using name_change
all_code_df.replace(name_change, inplace=True)
# all_code_df

,Country,CODE,Year,Value,COUNTRY,GDP (BILLIONS),country,latitude,longitude
0,Albania,ALB,14105,2103,Albania,13.40,AL,41.153332,20.168331
1,Australia,AUS,16116,719,Australia,1483.00,AU,-25.274398,133.775136
2,Austria,AUT,16116,53865,Austria,436.10,AT,47.516231,14.550072
3,Belgium,BEL,16116,23998,Belgium,527.80,BE,50.503887,4.469936
4,Bulgaria,BGR,16116,20986,Bulgaria,55.08,BG,42.733883,25.485830
5,Bosnia and Herzegovina,BIH,14105,333,Bosnia and Herzegovina,19.55,BA,43.915886,17.679076
6,Canada,CAN,16116,4748,Canada,1794.00,CA,56.130366,-106.346771
7,Switzerland,CHE,16116,16954,Switzerland,679.00,CH,46.818188,8.227512
8,Cyprus,CYP,16116,6972,Cyprus,21.34,CY,35.126413,33.429859
9,Czech Republic,CZE,16116,508,Czech Republic,205.60,CZ,49.817492,15.472962


In [15]:
# add coordinates for South Korea and Macedonia
all_code_df.at[24, 'latitude'] = 127.7669
all_code_df.at[24, 'longitude'] = 35.9078

all_code_df.at[29, 'latitude'] = 21.7453
all_code_df.at[29, 'longitude'] = 41.6086

# all_code_df

,Country,CODE,Year,Value,COUNTRY,GDP (BILLIONS),country,latitude,longitude
0,Albania,ALB,14105,2103,Albania,13.40,AL,41.153332,20.168331
1,Australia,AUS,16116,719,Australia,1483.00,AU,-25.274398,133.775136
2,Austria,AUT,16116,53865,Austria,436.10,AT,47.516231,14.550072
3,Belgium,BEL,16116,23998,Belgium,527.80,BE,50.503887,4.469936
4,Bulgaria,BGR,16116,20986,Bulgaria,55.08,BG,42.733883,25.485830
5,Bosnia and Herzegovina,BIH,14105,333,Bosnia and Herzegovina,19.55,BA,43.915886,17.679076
6,Canada,CAN,16116,4748,Canada,1794.00,CA,56.130366,-106.346771
7,Switzerland,CHE,16116,16954,Switzerland,679.00,CH,46.818188,8.227512
8,Cyprus,CYP,16116,6972,Cyprus,21.34,CY,35.126413,33.429859
9,Czech Republic,CZE,16116,508,Czech Republic,205.60,CZ,49.817492,15.472962


In [16]:
# was experimenting with removing the "trace 0" when hovering over countries
# removes rows where Country column has nan
dropped_nan_df = all_code_df[pd.notnull(all_code_df['Country'])]
# dropped_nan_df

In [22]:
# plot out the countries
data = [go.Choropleth(
    locations = all_code_df['CODE'],
    z = all_code_df['Value'],
    text = all_code_df['Country'],
    colorscale = [
        [0, "rgb(51, 0, 25)"],
        [0.3, "rgb(102, 0, 51)"],
        [0.5, "rgb(153, 0, 76)"],
        [0.6, "rgb(204, 0, 102)"],
        [0.7, "rgb(255, 51, 153)"],
        [1, "rgb(255, 153, 204)"]
    ],
    zmin = 0,
    zmax = 600000,
    autocolorscale = False,
    reversescale = True,
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(180,180,180)',
            width = 0.5
        )),
    colorbar = go.choropleth.ColorBar(
        tickprefix = '',
        title = 'Number of Syrian Refugees'),
)]

# following code is for lines connecting countries
countries = [go.Scattergeo(
    lon = dropped_nan_df['longitude'],
    lat = dropped_nan_df['latitude'],
    hoverinfo = 'none',
#     text = dropped_nan_df['COUNTRY'],
    mode = 'none',
#     marker = go.scattergeo.Marker(
#         size = 0,
#         color = 'rgb(255, 0, 0)',
#         line = go.scattergeo.marker.Line(
#             width = 3,
#             color = 'rgba(68, 68, 68, 0)'
#         ))
    )]

refugee_paths = []

syria_start_lon = 33.5138
syria_start_lat = 36.2765
maximum_value = float(dropped_nan_df['Value'].max())

for i in range(len(dropped_nan_df)):
    # setting the opacity of the lines
    opacity = 0
    country_refugee = dropped_nan_df['Value'][i]
    opacity = float(country_refugee) / maximum_value
    # doing this because otherwise, the lines to countries besides Germany are invisible
    if opacity < 0.25:
        opacity = 0.25
        
    refugee_paths.append(
        go.Scattergeo(
            locationmode = 'country names',
            lon = [syria_start_lon, dropped_nan_df['longitude'][i]],
            lat = [syria_start_lat, dropped_nan_df['latitude'][i]],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 1,
                color = 'red',
            ),
            opacity = opacity,
        )
    )

layout = go.Layout(
    showlegend = False,
    title = go.layout.Title(
        text = '2011-2018 Syrian Refugee Migration Patterns'
    ),
    
    geo = go.layout.Geo(
        showframe = False,
        showcoastlines = False,
        projection = go.layout.geo.Projection(
#             type = 'azimuthal equal area'
            type = 'equirectangular',
        ),
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)', 
    ),
    
    annotations = [go.layout.Annotation(
        x = 0.55,
        y = 0.1,
        xref = 'paper',
        yref = 'paper',
        text = 'Source: <a href="http://popstats.unhcr.org/en/asylum_seekers_monthly">\
            Asylum-Seekers (Monthly Data)</a>',
        showarrow = False
    )]
)

fig = go.Figure(data = data + refugee_paths + countries, layout = layout)
py.iplot(fig, validate = False, filename = 'd3-world-map')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~hwu43/0 or inside your plot.ly account where it is named 'd3-world-map'
